In [1]:
import math
import datetime
from memoize import *

In [2]:
def Time():
    print(str(datetime.datetime.now()))

In [3]:
class ContinuedFractionSystem:
    
    def __init__(self, knots, alphabet):
        self.knots = knots
        self.alphabet = [ float(x) for x in alphabet ]
        self.gamma = min(self.alphabet)
        
    @memoize
    def ContinuedFraction(self, x, m):
        return 1.0/(x + m)
    
    @memoize
    def ContinuedFractionDerivative(self, x, m):
        return -1.0/((x+m)**2.0)
    
    @memoize
    def BoundaryKnots(self, x):
        a = max([ knot for knot in self.knots if knot <= x ])
        b = min([ knot for knot in self.knots if knot >= x ])
        ai = self.knots.index(a)
        bi = self.knots.index(b)
        return (a,b,ai,bi)
    
    def Interpolate(self, v, x):
        (a,b,ai,bi) = self.BoundaryKnots(x)
        if ai == bi:
            return v[ai]
        return ((b-x)*v[ai] + (x-a)*v[bi])/(b-a)
    
    @memoize
    def LowerBoundError(self, s, x, m):
        # Formula on middle of page 8
        theta = self.ContinuedFraction(x,m)
        (a,b,ai,bi) = self.BoundaryKnots(theta)
        return (b-theta)*(theta-a)*s*(2.0*s + 1.0)*math.exp(2.0*s*(b-a)/self.gamma)/(self.gamma**2.0)
    
    @memoize
    def LowerCoefficients(self, s):
        return [ [ (abs(self.ContinuedFractionDerivative(x,m))**s) * (1.0 - self.LowerBoundError(s,x,m)) for m in self.alphabet ] for x in self.knots]

    def Lower(self, v, s):
        coefficients = self.LowerCoefficients(s)
        return [ sum( [coefficients[i][j] * self.Interpolate(v, self.ContinuedFraction(x,m)) \
                       for (j,m) in enumerate(self.alphabet)]) for (i,x) in enumerate(self.knots)]
    
    @memoize
    def UpperCoefficients(self, s):
        return [ [ abs(self.ContinuedFractionDerivative(x,m))**s for m in self.alphabet ] for x in self.knots]
    
    def Upper(self, v, s):
        coefficients = self.UpperCoefficients(s)
        return [ sum( [coefficients[i][j] * self.Interpolate(v, self.ContinuedFraction(x,m)) \
                       for (j,m) in enumerate(self.alphabet)]) for (i,x) in enumerate(self.knots)]

In [4]:
def norm(v):
    return sum([ abs(x) for x in v])

In [5]:
def normalize(v):
    k = norm(v)
    return [ x/k for x in v ]

In [6]:
def IsContractive(x0, F, tol=1E-16):
    N = len(x0)
    while 1:
        x = F(x0)
        if all( x[i] < x0[i] for i in range(0,N) ):
            return ("Contractive", x0)
        if all( x[i] > x0[i] for i in range(0,N) ):
            return ("Not Contractive", x0)
        if all( abs(x[i] - x0[i]) < tol for i in range(0,N) ):
            return ("Undetermined", x0)
        x0 = normalize(x)

In [7]:
def HausdorffDimensionBounds(continued_fraction_system):
    Lower = continued_fraction_system.Lower
    Upper = continued_fraction_system.Upper
    N = len(continued_fraction_system.knots)
    def BisectionSearch(F):
        def FIsContractive(s, x0 = None):
            if x0 == None:
                x0 = [1 for i in range(0, N)]
            return IsContractive(x0, lambda x : F(x, s))
        bottom = 0.0
        top = 1.0
        point = None
        while 1:
            s = (top + bottom)/2.0
            (status, point) = FIsContractive(s, point)
            if status == "Contractive":
                top = s
            elif status == "Not Contractive":
                bottom = s
            elif status == "Undetermined":
                break
            #print((bottom,top))
        return (bottom,top)
    lower_interval = BisectionSearch(Lower)
    upper_interval = BisectionSearch(Upper)
    #print("Lower interval = " + str(lower_interval))
    #print("Upper interval = " + str(upper_interval))
    return (lower_interval[0], upper_interval[1])

In [8]:
N = 10000
knots = [ float(i)/float(N) for i in range(0,N+1) ]
alphabet = [1.0,2.0]
continued_fraction_system = ContinuedFractionSystem(knots,alphabet)

In [9]:
Time()
print(HausdorffDimensionBounds(continued_fraction_system))
Time()

2017-04-04 11:57:38.354640
(0.5312805050925817, 0.5312805065404973)
2017-04-04 11:58:29.846697
